<a href="https://colab.research.google.com/github/sausis20/Youtube-Trending-Scraper-and-EDA/blob/main/combine_csv_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import glob
import pandas as pd
import requests

In [ ]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/YouTube Trending Scraper/output/")

In [ ]:
extension = 'csv'

#combine all LT files in the list
all_filenames = [i for i in glob.glob('*LT*.csv'.format(extension))]
df_combinedLT = pd.concat([pd.read_csv(f) for f in all_filenames ])

#combine all LV files in the list
all_filenames = [i for i in glob.glob('*LV*.csv'.format(extension))]
df_combinedLV = pd.concat([pd.read_csv(f) for f in all_filenames ])

#combine all EE files in the list
all_filenames = [i for i in glob.glob('*EE*.csv'.format(extension))]
df_combinedEE = pd.concat([pd.read_csv(f) for f in all_filenames ])

In [ ]:
cd '/content/gdrive/My Drive/Colab Notebooks/YouTube Trending Scraper/'

/content/gdrive/My Drive/Colab Notebooks/YouTube Trending Scraper


In [ ]:
with open('api_key.txt', 'r') as file:
    api_key = file.readline()

In [ ]:
#video categories for LT
video_categories_LT = requests.get(f'https://www.googleapis.com/youtube/v3/videoCategories?part=snippet&regionCode=LT&key={api_key}').json()
df_LT_cat = pd.json_normalize(video_categories_LT['items'])[['id','snippet.title']]

#video categories for LV
video_categories_LV = requests.get(f'https://www.googleapis.com/youtube/v3/videoCategories?part=snippet&regionCode=LV&key={api_key}').json()
df_LV_cat = pd.json_normalize(video_categories_LV['items'])[['id','snippet.title']]

#video categories for EE
video_categories_EE = requests.get(f'https://www.googleapis.com/youtube/v3/videoCategories?part=snippet&regionCode=EE&key={api_key}').json()
df_EE_cat = pd.json_normalize(video_categories_EE['items'])[['id','snippet.title']]

In [ ]:
df_LT_cat.head(5)

,id,snippet.title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [ ]:
#show differences between dataframes
print(df_LT_cat.compare(df_LV_cat))
print()
print(df_LT_cat.compare(df_EE_cat))

#This shows that all Baltic countries have the same Youtube video categories

Empty DataFrame
Columns: []
Index: []

Empty DataFrame
Columns: []
Index: []


In [ ]:
#change category data type and rename columns to more informative names
print(df_LT_cat.dtypes)
df_LT_cat['id']=df_LT_cat['id'].astype(int)

df_LT_cat = df_LT_cat.rename(columns={"id": "categoryId", "snippet.title": "Video Category"})

id               object
snippet.title    object
dtype: object


In [ ]:
df_LT = df_combinedLT.merge(df_LT_cat, on='categoryId', how='left')
df_LT['Country'] = 'LT'

df_LV = df_combinedLV.merge(df_LT_cat, on='categoryId', how='left')
df_LV['Country'] = 'LV'

df_EE = df_combinedEE.merge(df_LT_cat, on='categoryId', how='left')
df_EE['Country'] = 'EE'

In [ ]:
print(df_LT.shape, df_LV.shape, df_EE.shape)

(3800, 18) (3800, 18) (3800, 18)


In [ ]:
df = pd.concat([df_LT, df_LV, df_EE])

In [ ]:
df.shape

(11400, 12)

In [ ]:
df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,Video Category,Country
0,H8uGi4ZMgHA,Vaidas Baumila - Apžavai,2022-07-20T05:00:12Z,UCVmP73OeV2pCzBW6lita6gA,Vaidas Baumila,10,22.22.07,Vaidas Baumila|kunigunda|apžavai|apzavai|tavo ...,121809,0,0,118,https://i.ytimg.com/vi/H8uGi4ZMgHA/default.jpg,False,True,Vaidas Baumila - Apžavai Muzika ir tekstas - ...,Music,LT
1,0uAIEVI__iE,Nepatogūs klausimai. Svečiuose – Iglė Bernotai...,2022-07-17T16:00:19Z,UCvEFtIm2D5ePEBbKaAyvZ2w,Delfi,25,22.22.07,delfi|delfitv|laida|nepatogus klausimai|igle|i...,126097,0,0,129,https://i.ytimg.com/vi/0uAIEVI__iE/default.jpg,False,True,Iglė Bernotaitytė išgarsėjo prieš penkerius me...,News & Politics,LT
2,S4aTIcbeXlg,QUARTER-FINALS: France v Lithuania | Full Bask...,2022-07-21T18:48:21Z,UCtInrnU3QbWqFGsdKT1GZtg,FIBA - The Basketball Channel,17,22.22.07,21.07.2022|21/07/2022|FIBA|FIBA U20 European C...,124261,0,0,14,https://i.ytimg.com/vi/S4aTIcbeXlg/default.jpg,False,True,Watch the Quarter-Finals game between France a...,Sports,LT
3,4iRRvN23F08,Monique - Palauksiu kol užmigsi (Vertical Musi...,2022-07-15T06:00:11Z,UCCHySUOeQDI9BCEOSk1hKTQ,Monique,10,22.22.07,[none],125700,0,0,95,https://i.ytimg.com/vi/4iRRvN23F08/default.jpg,False,True,Koncertas su orkestru - https://www.bilietai.l...,Music,LT
4,_y2krec8POo,День сто сорок восьмой. Беседа с @Alexey Arest...,2022-07-21T19:44:50Z,UCQVtD_N4OeD-9PshBq7NwyQ,ФЕЙГИН LIVE,25,22.22.07,фейгин live|фейгин лайф|марк фейгин|mark feygi...,1436550,0,0,2943,https://i.ytimg.com/vi/_y2krec8POo/default.jpg,False,True,#Фейгин #ФейгинLIVE День сто сорок восьмой. Бе...,News & Politics,LT


In [ ]:
df = df.drop(['video_id', 'channelId', 'categoryId', 'tags', 'thumbnail_link', 'description'], axis=1)

In [ ]:
#export to csv
df.to_csv("Baltics_combined.csv", index=False, encoding='utf-8-sig')